In [1]:
!pip install pydub
!pip install acrcloud

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.2 MB/s eta 0:00:0000:0100:010m
  Created wheel for acrcloud: filename=acrcloud-14.10.2020-py3-none-any.whl size=1120 sha256=269146b6e097ab950711864db3a561483bb38e1f215094d79e7e9e929d292c07
  Stored in directory: /Users/datoapanta/Library/Caches/pip/wheels/98/21/3a/4d64594be595ea47ac84bd9fb61ed2cbe077cdc806c52d771b
Successfully built acrcloud


In [4]:
import os
import requests
import json
import time
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType

# ACRCloud credentials
acr_config = {
    'host': 'identify-eu-west-1.acrcloud.com',
    'access_key': '36a18fa34e82649bd029161e3e35b318',
    'access_secret': 'pOzlgKOTZsdehkkAHw05wdXkYcd0mvIgTpIw9vB5',
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO,
    'debug': False,
    'timeout': 10  # seconds
}

# Initialize ACRCloud recognizer
acr_recognizer = ACRCloudRecognizer(acr_config)

def recognize_song(file_path):
    try:
        result = acr_recognizer.recognize_by_file(file_path, 0, 30)
        result_dict = json.loads(result)
        if result_dict['status']['code'] == 0:
            title = result_dict['metadata']['music'][0]['title']
            artist = result_dict['metadata']['music'][0]['artists'][0]['name']
            return title, artist
        else:
            return None, None
    except Exception as e:
        print(f"Error recognizing song {file_path}: {e}")
        return None, None

def fetch_cover_image(title, artist):
    search_query = f"{title} {artist}"
    url = f"https://itunes.apple.com/search?term={search_query}&entity=song&limit=1"
    response = requests.get(url)
    data = response.json()
    if data['resultCount'] > 0:
        cover_url = data['results'][0]['artworkUrl100']
        return cover_url
    else:
        # Retry with a different service, such as Last.fm or Spotify
        return None

def download_cover_image(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

def process_directory(directory):
    results = []
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            file_path = os.path.join(directory, filename)
            title, artist = recognize_song(file_path)
            if title and artist:
                print(f"Title: {title}, Artist: {artist}")
                cover_url = fetch_cover_image(title, artist)
                if cover_url:
                    print(f"Cover Image URL: {cover_url}")
                    save_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}_cover.jpg")
                    download_cover_image(cover_url, save_path)
                    print(f"Cover image downloaded as '{save_path}'")
                else:
                    print(f"Cover image not found for {title} by {artist}.")
                results.append((filename, title, artist, cover_url))
            else:
                print(f"Song recognition failed for {filename}.")
            time.sleep(1)  # To avoid hitting rate limits
    return results

if __name__ == "__main__":
    directory = '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/blues'
    results = process_directory(directory)
    with open(os.path.join(directory, 'results.json'), 'w') as f:
        json.dump(results, f, indent=4)
    print("Processing completed. Results saved to 'results.json'.")


Title: Mermaids Flirt With Me, Artist: Hot Toddy
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music/69/4c/05/mzi.aijnhzkh.tif/100x100bb.jpg
Cover image downloaded as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/blues/blues.00093_cover.jpg'
Title: Right From The Start, Artist: Hot Toddy
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music/69/4c/05/mzi.aijnhzkh.tif/100x100bb.jpg
Cover image downloaded as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/blues/blues.00087_cover.jpg'
Title: What's Wrong, Artist: Magic Slim
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music124/v4/12/68/b5/1268b524-c4ee-5e94-29fd-0920096b5b49/mzi.ouyqvgby.jpg/100x100bb.jpg
Cover image downloaded as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/blues/blues.00050_cover.jpg'
Title: Testify, Artist: Stevie Ray Vaughan & Double Trouble
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Features125/v4/4c/30/3b/4c303b15-31ec-2f

In [5]:
import os
import requests
import json
import time
import pandas as pd
from PIL import Image
from io import BytesIO
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType

# ACRCloud credentials
acr_config = {
    'host': 'identify-eu-west-1.acrcloud.com',
    'access_key': '36a18fa34e82649bd029161e3e35b318',
    'access_secret': 'pOzlgKOTZsdehkkAHw05wdXkYcd0mvIgTpIw9vB5',
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO,
    'debug': False,
    'timeout': 10  # seconds
}

# Initialize ACRCloud recognizer
acr_recognizer = ACRCloudRecognizer(acr_config)

def recognize_song(file_path):
    try:
        result = acr_recognizer.recognize_by_file(file_path, 0, 30)
        result_dict = json.loads(result)
        if result_dict['status']['code'] == 0:
            title = result_dict['metadata']['music'][0]['title']
            artist = result_dict['metadata']['music'][0]['artists'][0]['name']
            return title, artist
        else:
            return None, None
    except Exception as e:
        print(f"Error recognizing song {file_path}: {e}")
        return None, None

def fetch_cover_image(title, artist):
    search_query = f"{title} {artist}"
    url = f"https://itunes.apple.com/search?term={search_query}&entity=song&limit=1"
    response = requests.get(url)
    data = response.json()
    if data['resultCount'] > 0:
        cover_url = data['results'][0]['artworkUrl100'].replace('100x100', '200x200')
        return cover_url
    else:
        return None

def download_and_resize_cover_image(url, save_path):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image = image.resize((200, 200))
    image.save(save_path)

def process_directory(directory):
    results = []
    for root, _, files in os.walk(directory):
        genre = os.path.basename(root)
        if genre in ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']:
            for filename in files:
                if filename.endswith(".wav"):
                    file_path = os.path.join(root, filename)
                    title, artist = recognize_song(file_path)
                    if title and artist:
                        print(f"Title: {title}, Artist: {artist}")
                        cover_url = fetch_cover_image(title, artist)
                        if cover_url:
                            print(f"Cover Image URL: {cover_url}")
                            genre_dir = os.path.join(directory, 'images', genre)
                            os.makedirs(genre_dir, exist_ok=True)
                            save_path = os.path.join(genre_dir, f"{os.path.splitext(filename)[0]}_cover.jpg")
                            download_and_resize_cover_image(cover_url, save_path)
                            print(f"Cover image downloaded and resized as '{save_path}'")
                        else:
                            print(f"Cover image not found for {title} by {artist}.")
                        results.append((filename, genre, title, artist, cover_url))
                    else:
                        print(f"Song recognition failed for {filename}.")
                    time.sleep(1)  # To avoid hitting rate limits
    return results

if __name__ == "__main__":
    base_directory = '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan'
    genres_directory = os.path.join(base_directory, 'genres')
    results = process_directory(genres_directory)
    
    # Save metadata to CSV
    metadata_path = os.path.join(base_directory, 'metadata.csv')
    df = pd.DataFrame(results, columns=['filename', 'label', 'title', 'artist', 'cover_url'])
    df.to_csv(metadata_path, index=False)
    
    print("Processing completed. Metadata saved to 'metadata.csv'.")


Title: Born to Make You Happy, Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music/0c/31/e2/mzi.hjqtzqwh.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/images/pop/pop.00027_cover.jpg'
Title: Joy Of Pepsi, Artist: Britney Spears
Cover image not found for Joy Of Pepsi by Britney Spears.
Title: Oops!...I Did It Again, Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music112/v4/c8/44/83/c8448346-cc2b-0e72-c912-9679d617cdfd/artwork.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/images/pop/pop.00032_cover.jpg'
Title: I'll Never Stop Loving You (2009 Remaster), Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music114/v4/8c/64/55/8c645545-fb91-7133-2cd8-a62b909c7f4b/dj.ktfmmnjq.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Des

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import os
import requests
import json
import time
import pandas as pd
from PIL import Image
from io import BytesIO
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType

# ACRCloud credentials
acr_config = {
    'host': 'identify-eu-west-1.acrcloud.com',
    'access_key': '36a18fa34e82649bd029161e3e35b318',
    'access_secret': 'pOzlgKOTZsdehkkAHw05wdXkYcd0mvIgTpIw9vB5',
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO,
    'debug': False,
    'timeout': 10  # seconds
}

# Initialize ACRCloud recognizer
acr_recognizer = ACRCloudRecognizer(acr_config)

def recognize_song(file_path):
    try:
        result = acr_recognizer.recognize_by_file(file_path, 0, 30)
        result_dict = json.loads(result)
        if result_dict['status']['code'] == 0:
            title = result_dict['metadata']['music'][0]['title']
            artist = result_dict['metadata']['music'][0]['artists'][0]['name']
            return title, artist
        else:
            return None, None
    except Exception as e:
        print(f"Error recognizing song {file_path}: {e}")
        return None, None

def fetch_cover_image(title, artist):
    search_query = f"{title} {artist}"
    url = f"https://itunes.apple.com/search?term={search_query}&entity=song&limit=1"
    response = requests.get(url)
    try:
        data = response.json()
        if data['resultCount'] > 0:
            cover_url = data['results'][0]['artworkUrl100'].replace('100x100', '200x200')
            return cover_url
        else:
            return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

def download_and_resize_cover_image(url, save_path):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image = image.resize((200, 200))
    image.save(save_path)

def process_directory(directory, processed_files):
    results = []
    errors = {}
    for root, _, files in os.walk(directory):
        genre = os.path.basename(root)
        if genre in ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']:
            for filename in files:
                if filename.endswith(".wav") and filename not in processed_files:
                    file_path = os.path.join(root, filename)
                    title, artist = recognize_song(file_path)
                    if title and artist:
                        print(f"Title: {title}, Artist: {artist}")
                        cover_url = fetch_cover_image(title, artist)
                        if cover_url:
                            try:
                                print(f"Cover Image URL: {cover_url}")
                                genre_dir = os.path.join(directory, 'images', genre)
                                os.makedirs(genre_dir, exist_ok=True)
                                save_path = os.path.join(genre_dir, f"{os.path.splitext(filename)[0]}_cover.jpg")
                                download_and_resize_cover_image(cover_url, save_path)
                                print(f"Cover image downloaded and resized as '{save_path}'")
                            except Exception as e:
                                print(f"Error downloading or resizing cover image for {title} by {artist}: {e}")
                                if genre not in errors:
                                    errors[genre] = []
                                errors[genre].append((filename, title, artist))
                        else:
                            print(f"Cover image not found for {title} by {artist}.")
                            if genre not in errors:
                                errors[genre] = []
                            errors[genre].append((filename, title, artist))
                        results.append((filename, genre, title, artist, cover_url))
                    else:
                        print(f"Song recognition failed for {filename}.")
                    time.sleep(1)  # To avoid hitting rate limits
    return results, errors

def load_existing_metadata(metadata_path):
    if os.path.exists(metadata_path):
        return pd.read_csv(metadata_path)
    else:
        return pd.DataFrame(columns=['filename', 'label', 'title', 'artist', 'cover_url'])

if __name__ == "__main__":
    base_directory = '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan'
    genres_directory = os.path.join(base_directory, 'genres')
    metadata_path = os.path.join(base_directory, 'metadata.csv')

    # Load existing metadata to avoid reprocessing
    existing_metadata = load_existing_metadata(metadata_path)
    processed_files = set(existing_metadata['filename'])

    # Process the directory and get new results
    new_results, errors = process_directory(genres_directory, processed_files)

    # Append new results to the existing metadata
    if new_results:
        new_metadata = pd.DataFrame(new_results, columns=['filename', 'label', 'title', 'artist', 'cover_url'])
        updated_metadata = pd.concat([existing_metadata, new_metadata], ignore_index=True)
        updated_metadata.to_csv(metadata_path, index=False)
    else:
        print("No new files to process.")

    # Save errors to separate CSV files
    for genre, error_list in errors.items():
        error_df = pd.DataFrame(error_list, columns=['filename', 'title', 'artist'])
        error_path = os.path.join(base_directory, 'errors', f'{genre}_errors.csv')
        os.makedirs(os.path.dirname(error_path), exist_ok=True)
        error_df.to_csv(error_path, index=False)

    print("Processing completed. Metadata updated.")


Title: Born to Make You Happy, Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music/0c/31/e2/mzi.hjqtzqwh.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/images/pop/pop.00027_cover.jpg'
Title: Joy Of Pepsi, Artist: Britney Spears
Cover image not found for Joy Of Pepsi by Britney Spears.
Title: Oops!...I Did It Again, Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music112/v4/c8/44/83/c8448346-cc2b-0e72-c912-9679d617cdfd/artwork.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Desktop/mma_2024/dataset/data/gtzan/genres/images/pop/pop.00032_cover.jpg'
Title: I'll Never Stop Loving You (2009 Remaster), Artist: Britney Spears
Cover Image URL: https://is1-ssl.mzstatic.com/image/thumb/Music114/v4/8c/64/55/8c645545-fb91-7133-2cd8-a62b909c7f4b/dj.ktfmmnjq.jpg/200x200bb.jpg
Cover image downloaded and resized as '/Users/datoapanta/Des